In [ ]:
!pip uninstall -y mlrun
!pip install git+https://github.com/mlrun/mlrun.git@development

In [1]:
# nuclio: ignore
# if the nuclio-jupyter package is not installed run !pip install nuclio-jupyter
import nuclio 

In [2]:
%nuclio cmd -c pip install pandas
%nuclio config spec.build.baseImage = "python:3.6-jessie"

%nuclio: setting spec.build.baseImage to 'python:3.6-jessie'


In [3]:
import os

def training(context, p1=1, p2=2):
    # access input metadata, values, and inputs
    print(f'Run: {context.name} (uid={context.uid})')
    print(f'Params: p1={p1}, p2={p2}')
    context.logger.info('started training')
    
    # do some training 
    
    # log the run results (scalar values)
    context.log_result('accuracy', p1 * 2)
    context.log_result('loss', p1 * 3)
    
    # add a lable/tag to this run 
    context.set_label('category', 'tests')
    
    # log a simple artifact + label the artifact 
    context.log_artifact('model.txt', body=b'abc is 123', labels={'framework': 'xgboost'})

def validation(context, model):
    # access input metadata, values, files, and secrets (passwords)
    print(f'Run: {context.name} (uid={context.uid})')
    #model = context.get_object('model', model)
    print('file - {}:\n{}\n'.format(model.url, model.get()))
    
    context.logger.info('started validation')
    
    context.log_artifact('validation.html', body=b'<b> validated </b>', viewer='web-app')

def listfiles(context, path='/'):
    l = os.listdir(path)
    print(l)
    return '{} contain {} files'.format(path, len(l))

## Load MLRUN and specify defaults 

In [4]:
# nuclio: end-code
# (end-code marker tells nuclio to stop parsing the notebook from this cell)

# set mlrun db path (can also be specified in run_start command)
%env MLRUN_DBPATH=/User/mlrun
#%env MLRUN_PACKAGE_PATH=git+https://github.com/mlrun/mlrun.git@development

from mlrun import new_function, code_to_function, NewRun
from mlrun.platforms import mount_v3io

env: MLRUN_DBPATH=/User/mlrun
env: MLRUN_PACKAGE_PATH=git+https://github.com/mlrun/mlrun.git@development


## Test the code locally
the functions above can be tested locally, parameters, inputs, and outputs can be specified in the API or the runspec object<br>
we create a `function` which defines the runtime environment (type, code, image, ..) and `run` a tasks/experiments using that function <br>
(we use the `local` runtime by default, later on we will use a `job` runtime for running containers, and can use other runners like MpiJob, Spark, Dask, Nuclio, ..)

in each run we can specify the function, inputs, parameters/hyper-parameters, etc. (check the `RunTemplate` class for details)<br>
in Jupyter runs print a summary table with metadata and links to data artifacts, this can be disabled with `visible=False` in `.run()`

In [5]:
fn = new_function()
list_run = fn.run(handler=listfiles, params={'path': '/User'})

[mlrun] 2019-09-16 07:27:29,140 starting run None uid=b2cb44b93b664de2bd8209dfad700a55
['.bash_profile', '.bashrc', '.config', '.igz', '.ipynb_checkpoints', '.ipython', '.jupyter', '.local', '.pythonlibs', '.viminfo', '.vimrc', '1', 'LICENSE', 'README.md', 'Untitled Folder', 'assets', 'demos', 'getting-started', 'igz-tutorials-get.sh', 'mlrun', 'update-tutorials.ipynb', 'v3io', 'welcome.ipynb']



uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
...700a55,0,Sep 16 07:27:29,completed,,kind=handlerowner=iguaziohost=jupyter-sm67srm2t3-w15ka-8bcf9dddd-mb2hs,,path=/User,return=/User contain 23 files,


type result.show() to see detailed results/progress or use CLI:
!mlrun get run --uid b2cb44b93b664de2bd8209dfad700a55 
[mlrun] 2019-09-16 07:27:29,290 run executed, status=completed


## Running and linking multiple tasks
in the next example we run two functions, `training` and `validation` and we pass the result from one to the other.<br>
we will see in the 'job' example that linking works even when the tasks run on different processes or containers, or in a workflow.

In [6]:
train_run = fn.run(handler=training, params={'p1': 5})
model_path = train_run.output('model.txt')
validation_run = fn.run(handler=validation, inputs={'model': model_path})

[mlrun] 2019-09-16 07:27:29,295 starting run None uid=3d6797e4e8e44b57ad5aede053cd1b4d
[mlrun] 2019-09-16 07:27:29,332 started training
Run:  (uid=3d6797e4e8e44b57ad5aede053cd1b4d)
Params: p1=5, p2=2



uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
...cd1b4d,0,Sep 16 07:27:29,completed,,kind=handlerowner=iguaziohost=jupyter-sm67srm2t3-w15ka-8bcf9dddd-mb2hscategory=tests,,p1=5,accuracy=10loss=15,model.txt


type result.show() to see detailed results/progress or use CLI:
!mlrun get run --uid 3d6797e4e8e44b57ad5aede053cd1b4d 
[mlrun] 2019-09-16 07:27:29,468 run executed, status=completed
[mlrun] 2019-09-16 07:27:29,469 starting run None uid=d0f8d72cc32545178efd0c565325c6f6
[mlrun] 2019-09-16 07:27:29,500 started validation
Run:  (uid=d0f8d72cc32545178efd0c565325c6f6)
file - model.txt:
b'abc is 123'




uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
...25c6f6,0,Sep 16 07:27:29,completed,,kind=handlerowner=iguaziohost=jupyter-sm67srm2t3-w15ka-8bcf9dddd-mb2hs,model,,,validation.html


type result.show() to see detailed results/progress or use CLI:
!mlrun get run --uid d0f8d72cc32545178efd0c565325c6f6 
[mlrun] 2019-09-16 07:27:29,595 run executed, status=completed


## Define cluster jobs and build images 
in order to use in a cluster we need to package our code and dependencies<br>
the `make_nuclio_job` call will automatically form a `Job` with list of dependencies and runtime configuration<br>
you can apply KubeFlow modifiers to configure resources like Volumes, `mount_v3io()` adds an iguazio v3io volume (Home of current user) to the job 

the `build()` command is optional it pre builds all the dependencies, so the runs will be faster. note the code and params can be updated per run

In [7]:
# create an ML function from the notebook, attache it to iguazio data fabric (v3io)
fn = code_to_function(runtime='job').apply(mount_v3io())

# prepare an image from the dependencies, so we wont need to build the image every run 
fn.build(image='mlrun/nuctest:latest')

[mlrun] 2019-09-16 07:28:03,331 building image (mlrun/nuctest:latest)
FROM python:3.6-jessie
WORKDIR /run
RUN pip install pandas
RUN pip install git+https://github.com/mlrun/mlrun.git@development
ENV PYTHONPATH /run
[mlrun] 2019-09-16 07:28:03,333 using in-cluster config.
[mlrun] 2019-09-16 07:28:03,349 Pod mlrun-build-j9bbw created
..
INFO[0000] Downloading base image python:3.6-jessie     
2019/09/16 07:28:05 No matching credentials were found, falling back on anonymous
INFO[0000] Unpacking rootfs as cmd RUN pip install pandas requires it. 
INFO[0010] Taking snapshot of full filesystem...        
INFO[0013] Skipping paths under /kaniko, as it is a whitelisted directory 
INFO[0013] Skipping paths under /empty, as it is a whitelisted directory 
INFO[0013] Skipping paths under /var/run, as it is a whitelisted directory 
INFO[0013] Skipping paths under /dev, as it is a whitelisted directory 
INFO[0013] Skipping paths under /proc, as it is a whitelisted directory 
INFO[0013] Skipping path

### Run the job on the cluster (build or use pre-built image)
note the listfiles call will return the same results as in the local run since the Job shares the same filesystem <br>
`with_code()` will inject the latest code to the job, in case we made changes (it doesnt require a new build)

In [8]:
fn.with_code().run(handler=listfiles, params={'path': '/User'})

[mlrun] 2019-09-16 07:29:35,050 starting run nuclio-jobs uid=a846485e78cd42a183be4a327c7c0095
[mlrun] 2019-09-16 07:29:35,076 using in-cluster config.
[mlrun] 2019-09-16 07:29:35,094 Pod nuclio-jobs-czlf8 created
.....
[mlrun] 2019-09-16 07:29:44,557 starting run nuclio-jobs uid=a846485e78cd42a183be4a327c7c0095
['.bash_profile', '.bashrc', '.config', '.igz', '.ipynb_checkpoints', '.ipython', '.jupyter', '.local', '.pythonlibs', '.viminfo', '.vimrc', '1', 'LICENSE', 'README.md', 'Untitled Folder', 'assets', 'demos', 'getting-started', 'igz-tutorials-get.sh', 'mlrun', 'update-tutorials.ipynb', 'v3io', 'welcome.ipynb']

type result.show() to see detailed results/progress or use CLI:
!mlrun get run --uid a846485e78cd42a183be4a327c7c0095 
[mlrun] 2019-09-16 07:29:44,676 run executed, status=completed


uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
...7c0095,0,Sep 16 07:29:44,completed,nuclio-jobs,kind=localowner=iguaziohost=nuclio-jobs-czlf8,,path=/User,return=/User contain 23 files,


type result.show() to see detailed results/progress or use CLI:
!mlrun get run --uid a846485e78cd42a183be4a327c7c0095 
[mlrun] 2019-09-16 07:29:45,186 run executed, status=completed


In [9]:
# define a run template, specify the artifacts output path, and add a label (can be used for search later)
run_base = NewRun(out_path='/User/mlrun/data').set_label('stage', 'dev')

In [10]:
# run our training task, with hyper params, and select the one with max accuracy
run = run_base.copy().with_hyper_params({'p1': [2,6,4]}, selector='max.accuracy')
train_run = fn.with_code().run(run, handler=training, name='my-training', params={'p1': 9})
model_path = train_run.output('model.txt')

[mlrun] 2019-09-16 07:30:10,549 starting run my-training uid=5b5e529fc4e04cdeb86037ad2f3f7aa3
[mlrun] 2019-09-16 07:30:10,590 Pod my-training-bf5gs created
..
[mlrun] 2019-09-16 07:30:15,692 starting run my-training uid=5b5e529fc4e04cdeb86037ad2f3f7aa3
[mlrun] 2019-09-16 07:30:15,789 started training
Run: my-training (uid=5b5e529fc4e04cdeb86037ad2f3f7aa3-1)
Params: p1=2, p2=2

[mlrun] 2019-09-16 07:30:15,884 started training
Run: my-training (uid=5b5e529fc4e04cdeb86037ad2f3f7aa3-2)
Params: p1=6, p2=2

[mlrun] 2019-09-16 07:30:15,974 started training
Run: my-training (uid=5b5e529fc4e04cdeb86037ad2f3f7aa3-3)
Params: p1=4, p2=2

type result.show() to see detailed results/progress or use CLI:
!mlrun get run --uid 5b5e529fc4e04cdeb86037ad2f3f7aa3 
[mlrun] 2019-09-16 07:30:16,143 run executed, status=completed


uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
...3f7aa3,0,Sep 16 07:30:15,completed,my-training,stage=devkind=localowner=iguazio,,p1=9,best_iteration=2accuracy=12loss=18,model.txtiteration_results.csv


type result.show() to see detailed results/progress or use CLI:
!mlrun get run --uid 5b5e529fc4e04cdeb86037ad2f3f7aa3 
[mlrun] 2019-09-16 07:30:21,265 run executed, status=completed


In [11]:
# running validation, use the best model result from the previos step 
fn.run(run_base, handler=validation, name='my-validation', inputs={'model': model_path})

[mlrun] 2019-09-16 07:30:21,270 starting run my-validation uid=c0526d61250b46eb9ffc1057eecd5dc1
[mlrun] 2019-09-16 07:30:21,316 Pod my-validation-2frpp created
..
[mlrun] 2019-09-16 07:30:25,720 starting run my-validation uid=c0526d61250b46eb9ffc1057eecd5dc1
[mlrun] 2019-09-16 07:30:25,813 started validation
Run: my-validation (uid=c0526d61250b46eb9ffc1057eecd5dc1)
file - /User/mlrun/data/2/model.txt:
b'abc is 123'


type result.show() to see detailed results/progress or use CLI:
!mlrun get run --uid c0526d61250b46eb9ffc1057eecd5dc1 
[mlrun] 2019-09-16 07:30:25,872 run executed, status=completed


uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
...cd5dc1,0,Sep 16 07:30:25,completed,my-validation,stage=devkind=localowner=iguaziohost=my-validation-2frpp,model,,,validation.html


type result.show() to see detailed results/progress or use CLI:
!mlrun get run --uid c0526d61250b46eb9ffc1057eecd5dc1 
[mlrun] 2019-09-16 07:30:30,978 run executed, status=completed


In [12]:
# list all jobs
!mlrun get po 

[mlrun] 2019-09-15 22:33:14,171 using in-cluster config.
state      started          type     name
Failed     Sep 14 00:11:40  job      mlrun-4zl7l
Succeeded  Sep 14 00:10:27  build    mlrun-build-2dlsb
Succeeded  Sep 14 00:28:29  build    mlrun-build-6tplm
Succeeded  Sep 15 22:30:36  build    mlrun-build-b4x2g
Succeeded  Sep 15 18:57:25  build    mlrun-build-bk46b
Succeeded  Sep 15 22:29:50  build    mlrun-build-brtpk
Succeeded  Sep 15 20:02:22  build    mlrun-build-gdz4q
Succeeded  Sep 14 11:30:30  build    mlrun-build-jc7hb
Succeeded  Sep 14 19:47:48  build    mlrun-build-mkdbn
Succeeded  Sep 13 23:47:27  build    mlrun-build-rqcpg
Succeeded  Sep 15 17:43:57  build    mlrun-build-xb8hc
Succeeded  Sep 15 22:24:04  build    mlrun-build-zbmx9
Succeeded  Sep 15 20:04:23  build    mlrun-build-zmlb4
Failed     Sep 14 00:13:56  job      mlrun-krlxm
Succeeded  Sep 14 00:29:43  job      mlrun-t94f6
Succeeded  Sep 14 10:14:14  job      my-training-h4qdj
Succeeded  Sep 15 20:06:27  job      my

In [13]:
# check job logs
!mlrun watch mlrun-vrzfx

[mlrun] 2019-09-07 22:34:02,037 using in-cluster config.

[mlrun] 2019-09-07 22:31:33,200 starting run None uid=4b56d99167e14e3294dad35a670ea3d1
['.bashrc', '.config', '.igz', '.ipynb_checkpoints', '.ipython', '.jupyter', '.kube', '.local', '.pythonlibs', '.viminfo', '.vimrc', 'LICENSE', 'README.md', 'assets', 'demos', 'getting-started', 'horovod', 'igz-tutorials-get.sh', 'mlrun', 'update-tutorials.ipynb', 'v3io', 'welcome.ipynb']

[mlrun] 2019-09-07 22:31:33,306 run executed, status=created
Pod mlrun-vrzfx last status is: succeeded
